In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *

In [2]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

# Will get traces from two videos:
btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)
#btf_fname_green = r'test2020-10-22_16-15-20_test4-channel-1-pco_camera2\test2020-10-22_16-15-20_test4-channel-1-pco_camera2bigtiff.btf'
#btf_fname_green = os.path.join(bigtiff_folder, btf_fname_green)

In [16]:
vol0 = get_single_volume(btf_fname_red, 0, num_slices=33, alpha=0.15)

In [17]:
alpha = 0.15
dat_foldername = r'..\point_cloud_alignment'
fname = os.path.join(dat_foldername, 'img100.tif')
f = lambda tif : (alpha*tif.asarray()).astype('uint8')
with tifffile.TiffFile(fname) as tif0:
    vol0_old = f(tif0)

## Use single function to track

In [20]:
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *


In [82]:
all_matches, all_conf, all_neurons = track_neurons_full_video(btf_fname_red,
                             start_frame=0,
                             num_frames=2,
                             num_slices=33,
                             alpha=0.15,
                             verbose=2)

Matching frames 0 and 1 (end at 2)
Finished 2 frames in 16.65183734893799 seconds


In [56]:
clust_df = build_tracklets_from_matches(all_neurons, all_matches, verbose=1)

0 / 999
1 / 999
Finished tracks [ 0  2  4  5 10 11 12 15 17 20 21 24 26 27 28 29 32 35 38 39 40 41 45 46
 47]
2 / 999
Finished tracks [ 0  1  2  4  5  6  7  8  9 10 11 12 13 15 16 17 20 21 24 25 26 27 28 29
 30 31 32 33 35 36 38 39 40 41 44 45 46 47 48 49 50 51 52 53 54 55 56 58
 59 60 63 64 65 66 68 69 70 71 72 73 75 77 78 79 82 84 86 89 92 94]
3 / 999
Finished tracks [  0   1   2   4   5   6   7   8   9  10  11  12  13  14  15  16  17  19
  20  21  23  24  25  26  27  28  29  30  31  32  33  34  35  36  38  39
  40  41  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59
  60  62  63  64  65  66  68  69  70  71  72  73  75  76  77  78  79  81
  82  84  85  86  89  91  92  93  94  95  96  97  98 108]
4 / 999
Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  38  39  40  41  43  44  45  46  47  48  49  50  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  6

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 239 240 241 242 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24

Finished tracks [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24

...

Finished tracks [    0     1     2 ... 16050 16052 16063]
691 / 999
Finished tracks [    0     1     2 ... 16088 16092 16094]
692 / 999
Finished tracks [    0     1     2 ... 16103 16111 16113]
693 / 999
Finished tracks [    0     1     2 ... 16135 16139 16140]
694 / 999
Finished tracks [    0     1     2 ... 16154 16159 16160]
695 / 999
Finished tracks [    0     1     2 ... 16173 16175 16177]
696 / 999
Finished tracks [    0     1     2 ... 16185 16189 16192]
697 / 999
Finished tracks [    0     1     2 ... 16207 16208 16212]
698 / 999
Finished tracks [    0     1     2 ... 16225 16228 16229]
699 / 999
Finished tracks [    0     1     2 ... 16246 16247 16249]
700 / 999
Finished tracks [    0     1     2 ... 16268 16272 16273]
701 / 999
Finished tracks [    0     1     2 ... 16282 16283 16285]
702 / 999
Finished tracks [    0     1     2 ... 16314 16317 16319]
703 / 999
Finished tracks [    0     1     2 ... 16319 16335 16344]
704 / 999
Finished tracks [    0     1     2 ... 16358 163

Finished tracks [    0     1     2 ... 17395 17397 17399]
754 / 999
Finished tracks [    0     1     2 ... 17417 17418 17421]
755 / 999
Finished tracks [    0     1     2 ... 17435 17436 17437]
756 / 999
Finished tracks [    0     1     2 ... 17457 17458 17460]
757 / 999
Finished tracks [    0     1     2 ... 17468 17470 17475]
758 / 999
Finished tracks [    0     1     2 ... 17502 17503 17506]
759 / 999
Finished tracks [    0     1     2 ... 17528 17529 17530]
760 / 999
Finished tracks [    0     1     2 ... 17533 17541 17542]
761 / 999
Finished tracks [    0     1     2 ... 17564 17565 17567]
762 / 999
Finished tracks [    0     1     2 ... 17583 17584 17586]
763 / 999
Finished tracks [    0     1     2 ... 17607 17608 17609]
764 / 999
Finished tracks [    0     1     2 ... 17618 17628 17629]
765 / 999
Finished tracks [    0     1     2 ... 17658 17665 17666]
766 / 999
Finished tracks [    0     1     2 ... 17682 17683 17690]
767 / 999
Finished tracks [    0     1     2 ... 17690 177

Finished tracks [    0     1     2 ... 18831 18834 18836]
817 / 999
Finished tracks [    0     1     2 ... 18841 18846 18848]
818 / 999
Finished tracks [    0     1     2 ... 18885 18887 18889]
819 / 999
Finished tracks [    0     1     2 ... 18905 18911 18914]
820 / 999
Finished tracks [    0     1     2 ... 18929 18932 18933]
821 / 999
Finished tracks [    0     1     2 ... 18949 18953 18955]
822 / 999
Finished tracks [    0     1     2 ... 18976 18978 18979]
823 / 999
Finished tracks [    0     1     2 ... 18996 18997 19001]
824 / 999
Finished tracks [    0     1     2 ... 19026 19027 19028]
825 / 999
Finished tracks [    0     1     2 ... 19045 19047 19049]
826 / 999
Finished tracks [    0     1     2 ... 19073 19074 19075]
827 / 999
Finished tracks [    0     1     2 ... 19100 19101 19103]
828 / 999
Finished tracks [    0     1     2 ... 19126 19127 19128]
829 / 999
Finished tracks [    0     1     2 ... 19147 19149 19150]
830 / 999
Finished tracks [    0     1     2 ... 19165 191

Finished tracks [    0     1     2 ... 20242 20246 20252]
880 / 999
Finished tracks [    0     1     2 ... 20266 20270 20275]
881 / 999
Finished tracks [    0     1     2 ... 20279 20280 20281]
882 / 999
Finished tracks [    0     1     2 ... 20315 20316 20318]
883 / 999
Finished tracks [    0     1     2 ... 20338 20339 20340]
884 / 999
Finished tracks [    0     1     2 ... 20346 20350 20356]
885 / 999
Finished tracks [    0     1     2 ... 20379 20382 20387]
886 / 999
Finished tracks [    0     1     2 ... 20397 20404 20406]
887 / 999
Finished tracks [    0     1     2 ... 20411 20416 20419]
888 / 999
Finished tracks [    0     1     2 ... 20432 20433 20439]
889 / 999
Finished tracks [    0     1     2 ... 20457 20460 20461]
890 / 999
Finished tracks [    0     1     2 ... 20469 20471 20482]
891 / 999
Finished tracks [    0     1     2 ... 20505 20507 20508]
892 / 999
Finished tracks [    0     1     2 ... 20527 20528 20529]
893 / 999
Finished tracks [    0     1     2 ... 20555 205

Finished tracks [    0     1     2 ... 21623 21625 21626]
943 / 999
Finished tracks [    0     1     2 ... 21648 21651 21652]
944 / 999
Finished tracks [    0     1     2 ... 21669 21672 21673]
945 / 999
Finished tracks [    0     1     2 ... 21692 21693 21696]
946 / 999
Finished tracks [    0     1     2 ... 21714 21716 21717]
947 / 999
Finished tracks [    0     1     2 ... 21737 21738 21740]
948 / 999
Finished tracks [    0     1     2 ... 21754 21764 21766]
949 / 999
Finished tracks [    0     1     2 ... 21780 21781 21782]
950 / 999
Finished tracks [    0     1     2 ... 21802 21803 21805]
951 / 999
Finished tracks [    0     1     2 ... 21816 21820 21821]
952 / 999
Finished tracks [    0     1     2 ... 21836 21843 21848]
953 / 999
Finished tracks [    0     1     2 ... 21871 21874 21875]
954 / 999
Finished tracks [    0     1     2 ... 21889 21890 21893]
955 / 999
Finished tracks [    0     1     2 ... 21907 21910 21917]
956 / 999
Finished tracks [    0     1     2 ... 21937 219

## Save

In [57]:
import pickle

In [58]:
fname = 'clust_df_dat.pickle'
pickle.dump(clust_df,open(fname,'wb'))

In [59]:
fname = 'match_dat.pickle'
pickle.dump((all_matches, all_conf, all_neurons), open(fname,'wb'))

## Visualize

In [84]:
i=0

neurons0 = all_neurons[i]
neurons1 = all_neurons[i+1]
matches = all_matches[i]

visualize_tracks(neurons0, neurons1, matches)

In [77]:
all_conf[400]

[0.3333333333333333,
 0.6176470588235294,
 0.39285714285714285,
 0.6086956521739131,
 0.43478260869565216,
 0.3157894736842105,
 0.4878048780487805,
 0.5925925925925926,
 0.6388888888888888,
 0.47058823529411764,
 0.48,
 0.3,
 0.66,
 0.48,
 0.5254237288135594,
 0.5882352941176471,
 0.4642857142857143,
 0.6216216216216216,
 0.5932203389830508,
 0.2857142857142857,
 0.375,
 0.4782608695652174,
 0.25,
 0.27906976744186046,
 0.5454545454545454,
 0.3333333333333333,
 0.375,
 0.5423728813559322,
 0.2857142857142857,
 0.5263157894736842,
 0.5454545454545454,
 0.39285714285714285,
 0.35714285714285715,
 0.5535714285714286,
 0.7796610169491526,
 0.7391304347826086,
 0.4745762711864407,
 0.5757575757575758,
 0.4,
 0.5,
 0.7796610169491526,
 0.559322033898305,
 0.68,
 0.576271186440678,
 0.36363636363636365,
 0.6428571428571429,
 0.2916666666666667,
 0.5238095238095238,
 0.32142857142857145,
 0.5128205128205128,
 0.6440677966101694,
 0.423728813559322,
 0.6481481481481481,
 0.42105263157894735,
 

# Aside: timing tests

In [15]:
# Open video
def open_tif(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        1+1

def open_page(vid_fname):
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat = page.asarray()
            if i>0:
                break

def open_volume(vid_fname, num_slices=33):
    dat = []
    with tifffile.TiffFile(vid_fname) as tif:
        for i, page in enumerate(tif.pages):
            dat.append(page.asarray())
            if i>num_slices:
                break
                
def open_with_imread(vid_fname, num_slices=33):
    dat = tifffile.imread(vid_fname, key=range(num_slices))

In [9]:
%timeit open_tif(btf_fname_red)

269 µs ± 4.12 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%timeit open_page(btf_fname_red)

2 ms ± 32.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%timeit open_volume(btf_fname_red)

38.8 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit open_with_imread(btf_fname_red)

31.5 ms ± 4.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
